In [ ]:
#Sistema
import os,sys,inspect

import requests     #Traer el fichero de internet
import json         # Trabaja con ficheros json
import pandas as pd # Librería para el data mining y data wrangling
import numpy as np 

# visualización
from skimage.io import imread
import matplotlib.pyplot as plt
from matplotlib.pyplot import *
import scipy.stats as stats
import seaborn as sns
from mpl_toolkits.axes_grid1 import make_axes_locatable

"""
    xmldict
    ~~~~~~~~~~~~~~~~~~~~~~~~~
    Convert xml to python dictionaries.
"""
import datetime

def xml_to_dict(root_or_str, strict=True):
    """
    Converts `root_or_str` which can be parsed xml or a xml string to dict.
    """
    root = root_or_str
    if isinstance(root, str):
        import xml.etree.cElementTree as ElementTree
        root = ElementTree.XML(root_or_str)
    return {root.tag: _from_xml(root, strict)}

def dict_to_xml(dict_xml):
    """
    Converts `dict_xml` which is a python dict to corresponding xml.
    """
    return _to_xml(dict_xml)

# Functions below this line are implementation details.
# Unless you are changing code, don't bother reading.
# The functions above constitute the user interface.

def _to_xml(el):
    """
    Converts `el` to its xml representation.
    """
    val = None
    if isinstance(el, dict):
        val = _dict_to_xml(el)
    elif isinstance(el, bool):
        val = str(el).lower()
    else:
        val = el
    if val is None: val = 'null'
    return val

def _extract_attrs(els):
    """
    Extracts attributes from dictionary `els`. Attributes are keys which start
    with '@'
    """
    if not isinstance(els, dict):
        return ''
    return ''.join(' %s="%s"' % (key[1:], value) for key, value in els.iteritems()
                   if key.startswith('@'))

def _dict_to_xml(els):
    """
    Converts `els` which is a python dict to corresponding xml.
    """
    def process_content(tag, content):
        attrs = _extract_attrs(content)
        text = isinstance(content, dict) and content.get('#text', '') or ''
        return '<%s%s>%s%s</%s>' % (tag, attrs, _to_xml(content), text, tag)

    tags = []
    for tag, content in els.iteritems():
        # Text and attributes
        if tag.startswith('@') or tag == '#text' or tag == '#value':
            continue
        elif isinstance(content, list):
            for el in content:
                tags.append(process_content(tag, el))
        elif isinstance(content, dict):
            tags.append(process_content(tag, content))
        else:
            tags.append('<%s>%s</%s>' % (tag, _to_xml(content), tag))
    return ''.join(tags)

def _str_to_datetime(date_str):
    try:
        val = datetime.datetime.strptime(date_str,  "%Y-%m-%dT%H:%M:%SZ")
    except ValueError:
        val = date_str
    return val

def _str_to_boolean(bool_str):
    if bool_str.lower() != 'false' and bool(bool_str):
        return True
    return False

def _from_xml(el, strict):
    """
    Extracts value of xml element element `el`.
    """
    val = None
    # Parent node.
    if len(el):
        val = {}
        for e in el:
            tag = e.tag
            v = _from_xml(e, strict)
            if tag in val:
                # Multiple elements share this tag, make them a list
                if not isinstance(val[tag], list):
                    val[tag] = [val[tag]]
                val[tag].append(v)
            else:
                # First element with this tag
                val[tag] = v
    # Simple node.
    else:
        attribs = el.items()
        # An element with attributes.
        if attribs and strict:
            val = dict(('@%s' % k, v) for k, v in dict(attribs).iteritems())
            if el.text:
                converted = _val_and_maybe_convert(el)
                val['#text'] = el.text
                if converted != el.text:
                    val['#value'] = converted
        elif el.text:
            # An element with no subelements but text.
            val = _val_and_maybe_convert(el)
        elif attribs:
            val = dict(attribs)
    return val

def _val_and_maybe_convert(el):
    """
    Converts `el.text` if `el` has attribute `type` with valid value.
    """
    text = el.text.strip()
    data_type = el.get('type')
    convertor = _val_and_maybe_convert.convertors.get(data_type)
    if convertor:
        return convertor(text)
    else:
        return text
_val_and_maybe_convert.convertors = {
    'boolean': _str_to_boolean,
    'datetime': _str_to_datetime,
    'integer': int
}


if __name__ == "__main__":
    
    #currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))  --> otra forma de encontrar el directorio del archivo, por favor, explicar diferencia.
    currentdir = os.getcwd()            #path del fichero en jupiter
    path = os.path.dirname(__file__)    #path del fichero en .py
    parentdir = os.path.dirname(currentdir) # subir un nivel de directorios
    sys.path.insert(0,parentdir)        #insertar el directorio en el path (necesario para poder importar librerías con 'import')

    # para importar una imagen hay que ir directamente al archivo; os.sep inserta los separadores del sistema operativo correspondiente
    image = Image.open(path + os.sep + 'img' + os.sep + 'happy.jpg')

    def get_root_path(n):
        path = os.getcwd() # para notebook ||| __file__ --> para .py
        for i in range(n):
            path = os.path.dirname(path)
        print(path)
        sys.path.append(path)

    # n --> es el número de veces que se tiene que hacer os.path.dirname 
    # si en .ipynb notebook es 5 entonces en .py usando __file__ es 6. 
    get_root_path(n=5)

    # Se importan las librerías de clases desde un directorio superior
    import objects.human as ohu
    import objects.ork as oo
